<img src="https://www.exegetic.biz/img/exegetic-banner-black.svg" width="35%" align="right">

# Web Scraping: Members of Parliament

Andrew B. Collier (@datawookie | andrew@exegetic.biz)<br>
Data Scientist / Founder<br>
[Exegetic Analytics](https://www.exegetic.biz)

<span style="color: #3498db;">**↯ Notebooks**</span> available from https://bit.ly/2kwWRvX.

## Introduction

In this tutorial we're going to scrape (public) details of our esteemed members of parliament from the website of the [Parliamentary Monitoring Group](https://pmg.org.za/).

![](../fig/members-of-parliament.png)

**The Brief**: Our brief is to capture data for all members and store it in a relational database. Why? Well, suppose you were developing an insurance or investment product targeted specifically at politicians, then this would immediately give you a list of prospects with their contact details.

**The Challenge**: There's an index page with links to individual pages for each of the members. Need to systematically scrape all of the member pages.

**The Approach:** These are the steps that we'll take to achieve that goal:

1. Manually scrape the data for a specific member.
2. Write a function to scrape the data for a specific member.
3. Test that function.
4. Run the function across all of the members.
5. Store the results.

## Setup

Load some libraries.

In [ ]:
suppressMessages(library(dplyr))                           # Data wrangling
library(tidyr)                                             # More data wrangling
library(purrr)                                             # Functional tools
library(stringr)                                           # Working with strings
library(tictoc)                                            # Timing
library(RSQLite)                                           # SQLite databases
suppressMessages(library(rvest))                           # Scraping (static) websites

Synchronise your watches (or your RNGs).

In [ ]:
set.seed(17)

The name of the SQLite database that we'll use to store the data.

In [ ]:
SQLITEDB = "members-of-parliament.sqlite"

The base URL. Open [this link](https://pmg.org.za/members/) in your browser. The page is an index of the members, with a thumbnail linking to their individual profile pages.

In [ ]:
URL = "https://pmg.org.za/members/"

## Manual Scrape

Grab the HTML for a specific member's page.

In [ ]:
(person <- read_html("https://www.pa.org.za/person/alexandra-lilian-amelia-abrahams/"))

Start by retrieving the person's name. Need to get the appropriate CSS selector. In this case it's easy: it's the only `<h1>` tag on the page.

In [ ]:
person %>%
    html_node("h1") %>%                                    # Select specific tag
    html_text()                                            # Grab text enclosed by tag

<span style="color: #3498db;">**↯ Exercise**</span> Raw scraped data are often grubby. Clean it by using `str_squish()` to remove excess whitespace.

In [ ]:
# ------------------------------------------------------------------------------
#
# Your code goes here.
#
# ------------------------------------------------------------------------------

In [ ]:
person %>%
    html_node("h1") %>%
    html_text() %>%
    str_squish()                                           # Strip off whitespace

Next let's get party affiliation. This information is in a `<a>` tag but it's the only tag on the page which has the `party-membership--party` class.

In [ ]:
person %>%
    html_node(".party-membership--party") %>%
    html_text()

Now let's get the email address. The address is in a `<a>` tag nested inside a `<span>` with class `email-address`.

In [ ]:
person %>%
    html_nodes(".email-address a") %>%
    html_text()

<span style="color: #3498db;">**↯ Exercise**</span> Use `str_c()` to concatenate multiple email addresses with a semicolon separator.

In [ ]:
# ------------------------------------------------------------------------------
#
# Your code goes here.
#
# ------------------------------------------------------------------------------

In [ ]:
person %>%
    html_nodes(".email-address a") %>%
    html_text() %>% str_c(collapse = "; ")

This is good progress, but if we want to do this systematically across all members then we'll need to write a *function*.

## Scraping Function

The function should accept an URL and return a data frame with the scraped data.

In [ ]:
get_person <- function(url) {  
  # Deal with possible 404 errors.
  #
  person <- possibly(read_html, otherwise = NULL)(url)

  if (is.null(person)) {
    NULL
  } else {
    # Handle missing phone number and concatenate if multiple entries.
    phone = person %>% html_nodes('[href^="tel:"]')
    phone = ifelse(length(phone), phone %>% html_text() %>% str_c(collapse = "; "), NA)
    # Create data frame with results.
    tibble(
      name = person %>% html_node("h1") %>% html_text() %>% str_squish(),
      party = person %>% html_node(".party-membership--party") %>% html_text(),
      phone,
      email = person %>% html_nodes(".email-address a") %>% html_text() %>% str_c(collapse = "; ")
    )
  }
}

Let's run a few quick tests on the following members:

- [Alexandra Lilian Amelia Abrahams](https://www.pa.org.za/person/alexandra-lilian-amelia-abrahams/)
- [Rachel Cecilia Adams](https://www.pa.org.za/person/rachel-cecilia-adams/) and
- [Mr Michael Bagraim](https://www.pa.org.za/person/michael-bagraim/).

In [ ]:
get_person("https://www.pa.org.za/person/alexandra-lilian-amelia-abrahams/")

In [ ]:
get_person("https://www.pa.org.za/person/rachel-cecilia-adams/")

In [ ]:
get_person("https://www.pa.org.za/person/michael-bagraim/")

Those all look good. I think we're ready to start scraping at scale!

## Scraping All Members

First get the HTML for the index page.

In [ ]:
directory <- read_html(URL)

Extract all of the URLs for members' pages. These URLs are in `<div>` tags with `single-mp` class.

In [ ]:
parliament <- tibble(
  url = directory %>%
    html_nodes(".single-mp") %>%                           # Get all of the cards
    # Interate over cards.
    map_chr(function(member) {
      member %>%
        html_node("a") %>%                                 # Get the <a> tag
        html_attr("href")                                  # Extract the href attribute
    })
)

In [ ]:
# How many links?
#
nrow(parliament)

In [ ]:
# Take a look at the first few links.
#
head(parliament, 10)

Keep only URLs which are on <https://www.pa.org.za/>.

In [ ]:
parliament <- parliament %>% filter(str_detect(url, "^https://www.pa.org.za/"))

Now iterate over a random subset of URLs, scraping each one in turn.

In [ ]:
tic()
#
members <- parliament %>%
  sample_n(20) %>%
  mutate(
    data = map(url, get_person)
  )
#
toc()

<span style="color: #3498db;">**↯ Exercise**</span> Make the code above a little more server-friendly by introducing a delay. *Hint:* Use `Sys.sleep()` to pause and `rpois()` to sample a random number of seconds.

In [ ]:
# ------------------------------------------------------------------------------
#
# Your code goes here.
#
# ------------------------------------------------------------------------------

In [ ]:
tic()
#
members <- parliament %>%
  sample_n(20) %>%
  mutate(
    data = map(
        url,
        function(url) {
            Sys.sleep(rpois(1, 5))
            get_person(url)
        })
  )
#
toc()

Drop records without data.

In [ ]:
members <- members %>% filter(
    !sapply(members$data, is.null)
)

The `data` column is a "list column" containing nested data frames. Unpack it.

In [ ]:
members <- members %>% select(-url) %>% unnest()
head(members)

So there we have the contact details of members of parliament.

Parliament is by no means static. Members come and go. Since we have a script though, we just have to run the script again to update the data.

## Database

To finish off we'll save the data to a [SQLite](https://www.sqlite.org/index.html) database. First let's add an ID column to the table.

In [ ]:
members <- members %>%
    mutate(id = row_number()) %>%
    select(id, everything())

head(members)

Now write to a SQLite database.

In [ ]:
db <- dbConnect(RSQLite::SQLite(), SQLITEDB)

Write the data frame as a table. Write over the table if it already exists.

In [ ]:
dbWriteTable(db, "members", members, overwrite = TRUE)

Finally close the connection to the database.

In [ ]:
dbDisconnect(db)

It'd be good to check on the content of the database. You can download a local copy as follows:

- select File ⟶ Open;
- check the box next to the file you've just created; and
- press the Download button.

You can open the file with something like [DB Browser for SQLite](https://sqlitebrowser.org/).